In [1]:
import os, sys
import time
import subprocess
import glob
from pprint import pprint
import numpy as np
from numpy.linalg import norm
import itertools 
import re
import pandas as pd
import shutil

root_dir = os.getcwd() + '/..'
sys.path.append(root_dir)

from orca_wrapper import run_orca, parse_orca_output, default_search_terms, get_imaginary_modes

os.chdir('Orca_Calculations/')

# Step 1 - Optimize Non-interacting Geometries

In [2]:
input_files = glob.glob('Step1/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]
output_files

['Step1/Br-.out', 'Step1/Cl-.out', 'Step1/CH3Br.out', 'Step1/CH3Cl.out']

In [3]:
for i_file in input_files:
    run_orca(i_file)


Running "Step1/Br-.inp"
Writing to "Step1/Br-.out"

Running "Step1/Cl-.inp"
Writing to "Step1/Cl-.out"

Running "Step1/CH3Br.inp"
Writing to "Step1/CH3Br.out"

Running "Step1/CH3Cl.inp"
Writing to "Step1/CH3Cl.out"


# Step 2 - Optimize Weakly Interacting Geometries

In [4]:
# Use the optimized geometry for CH3Br as a starting point for our sweep
with open('Step1/CH3Br.xyz') as init_xyz:
    lines = init_xyz.readlines()
    nbr_atoms_before = lines[0].strip()
    nbr_atoms_after = 6
    lines[0] = f'{nbr_atoms_after}\n'
    lines.append('  Cl  -3.7                  0.0                    0.0\n')

for line in lines:
    print(line, end='')

with open('Step2/Cl-_CH3Br_init.xyz', 'w') as new_xyz:
    new_xyz.write(''.join(lines))

# Use the optimized geometry for CH3Cl as a starting point for our sweep
with open('Step1/CH3Cl.xyz') as init_xyz:
    lines = init_xyz.readlines()
    nbr_atoms_before = lines[0].strip()
    nbr_atoms_after = 6
    lines[0] = f'{nbr_atoms_after}\n'
    lines.append('  Br  -3.7                  0.0                    0.0\n')

for line in lines:
    print(line, end='')

with open('Step2/CH3Cl_Br-_init.xyz', 'w') as new_xyz:
    new_xyz.write(''.join(lines))

6
Coordinates from ORCA-job Step1/CH3Br
  C   -0.00128235262355     -0.00000008866415     -0.00000004468743
  Br  1.95252041825261     -0.00000089889325     -0.00000011758461
  H   -0.34394901973682      1.04129517371190      0.00477118086097
  H   -0.34394724288329     -0.52477881212232      0.89940066535969
  H   -0.34394699150895     -0.51651543043218     -0.90417168634862
  Cl  -3.7                  0.0                    0.0
6
Coordinates from ORCA-job Step1/CH3Cl
  C   -0.00625646210719      0.00000098892853     -0.00000010741695
  Cl  1.78918679828360     -0.00000273015495      0.00000009314880
  H   -0.36235932791891      1.03838548752991      0.00475773238384
  H   -0.36235393511022     -0.52331198592695      0.89688273294590
  H   -0.36235418464728     -0.51507161247654     -0.90164043336158
  Br  -3.7                  0.0                    0.0


In [5]:
input_files = glob.glob(f'Step2/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]
output_files

['Step2/Cl-_CH3Br.out', 'Step2/CH3Cl_Br-.out']

In [6]:
for i_file in input_files:
    run_orca(i_file)


Running "Step2/Cl-_CH3Br.inp"
Writing to "Step2/Cl-_CH3Br.out"

Running "Step2/CH3Cl_Br-.inp"
Writing to "Step2/CH3Cl_Br-.out"


# Step 3 - Scan for Transition State

In [7]:
shutil.copy('Step2/Cl-_CH3Br.xyz', 'Step3/Cl-_CH3Br_init.xyz')

'Step3/Cl-_CH3Br_init.xyz'

In [8]:
input_files = glob.glob('Step3/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]
output_files

In [9]:
for i_file in input_files:
    run_orca(i_file)


Running "Step3/Cl-_CH3Br.inp"
Writing to "Step3/Cl-_CH3Br.out"


In [10]:
for o_file in output_files:
    # get the highest energy transition state
    with open(o_file, 'r') as of:
        content = of.read()
        split_str = "The Calculated Surface using the 'Actual Energy'\n"
        trans_state_values = content.split(split_str)[1].split('\n\n',1)[0].split('\n')
    print(trans_state_values)
    energies = [float(s.strip().split()[1]) for s in trans_state_values ]

pprint(energies)

ts_index = np.argmax(energies) + 1
print(f'\nTransition state supected at index {ts_index}')

['   2.90000000 -3073.63538623', '   2.70000000 -3073.63446451', '   2.50000000 -3073.63260691', '   2.30000000 -3073.63602085', '   2.10000000 -3073.64416383', '   1.90000000 -3073.65156792']
[-3073.63538623,
 -3073.63446451,
 -3073.63260691,
 -3073.63602085,
 -3073.64416383,
 -3073.65156792]

Transition state supected at index 3


# Step 4 - Find Imaginary Modes

In [11]:
shutil.copy(f'Step3/Cl-_CH3Br.{ts_index:03}.xyz', 'Step4/Cl-_CH3Br_TS_init.xyz')

'Step4/Cl-_CH3Br_TS_init.xyz'

In [12]:
input_files = glob.glob(f'Step4/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]
output_files

['Step4/[Cl_CH3_Br]-.out']

In [13]:
for i_file in input_files:
    run_orca(i_file)


Running "Step4/[Cl_CH3_Br]-.inp"
Writing to "Step4/[Cl_CH3_Br]-.out"


In [14]:
im_freq_lines = get_imaginary_modes(output_files[0])
im_freqs = []
im_freq_idx = []
for line in im_freq_lines:
    idx_str, freq_str, _ = line.split(None, 2)
    idx = int(idx_str[:-1])
    freq = float(freq_str)
    if idx not in im_freq_idx:
        im_freq_idx.append(idx)
        im_freqs.append(freq)

if len(im_freq_idx) > 1:
    print("More than one imginary frequency found! This is not a transition state!")
    print(im_freq_lines)
elif len(im_freq_idx) < 1:
    print("No imgaginary frequencies found!")
else:
    reaction_idx = im_freq_idx[0]
    print(f"Reaction coordinate given by mode {reaction_idx} with imaginary frequency {im_freqs[0]} cm**-1")

Reaction coordinate given by mode 6 with imaginary frequency -282.42 cm**-1


In [15]:
hess_file = [i_file.replace('.inp', '.hess') for i_file in input_files]
print(hess_file[0])
p_out = subprocess.run(['orca_pltvib', hess_file[0], str(reaction_idx)], capture_output=True)
if p_out.stderr:
    print(p_out.stderr.decode())
print(p_out.stdout.decode())

Step4/[Cl_CH3_Br]-.hess
The file is being recognized as a valid orca hessian file
Will plot 1 vibrations:
   6
creating: Step4/[Cl_CH3_Br]-.hess.v006.xyz



# Step 5 - Single Point Calculations with Bigger Basis

In [181]:
input_files = glob.glob('Step5/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]
output_files

['Step5/Br-.out',
 'Step5/Cl-.out',
 'Step5/Cl-_CH3Br.out',
 'Step5/CH3Br.out',
 'Step5/[Cl_CH3_Br]-.out',
 'Step5/CH3Cl_Br-.out',
 'Step5/CH3Cl.out']

In [182]:
for i_file in input_files:
    run_orca(i_file)


Running "Step5/Br-.inp"
Writing to "Step5/Br-.out"

Running "Step5/Cl-.inp"
Writing to "Step5/Cl-.out"

Running "Step5/Cl-_CH3Br.inp"
Writing to "Step5/Cl-_CH3Br.out"

Running "Step5/CH3Br.inp"
Writing to "Step5/CH3Br.out"

Running "Step5/[Cl_CH3_Br]-.inp"
Writing to "Step5/[Cl_CH3_Br]-.out"

Running "Step5/CH3Cl_Br-.inp"
Writing to "Step5/CH3Cl_Br-.out"

Running "Step5/CH3Cl.inp"
Writing to "Step5/CH3Cl.out"


# Step 6 - Solvation Effects

In [183]:
input_files = glob.glob('Step6/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]
output_files

['Step6/Cl-_CH3Br.out', 'Step6/[Cl_CH3_Br]-.out', 'Step6/CH3Cl_Br-.out']

In [184]:
for i_file in input_files:
    run_orca(i_file)


Running "Step6/Cl-_CH3Br.inp"
Writing to "Step6/Cl-_CH3Br.out"

Running "Step6/[Cl_CH3_Br]-.inp"
Writing to "Step6/[Cl_CH3_Br]-.out"

Running "Step6/CH3Cl_Br-.inp"
Writing to "Step6/CH3Cl_Br-.out"


# Putting Together the Results

In [186]:
output_files = glob.glob('Step*/*.out')
output_files

['Step4/[Cl_CH3_Br]-.out',
 'Step3/Cl-_CH3Br.out',
 'Step2/Cl-_CH3Br.out',
 'Step2/CH3Cl_Br-.out',
 'Step5/CH3Br.out',
 'Step5/Cl-_CH3Br.out',
 'Step5/Cl-.out',
 'Step5/Br-.out',
 'Step5/CH3Cl_Br-.out',
 'Step5/[Cl_CH3_Br]-.out',
 'Step5/CH3Cl.out',
 'Step6/Cl-_CH3Br.out',
 'Step6/CH3Cl_Br-.out',
 'Step6/[Cl_CH3_Br]-.out',
 'Step1/CH3Br.out',
 'Step1/Cl-.out',
 'Step1/Br-.out',
 'Step1/CH3Cl.out']

In [187]:
calcs = {}
for o_file in output_files:
    terms = parse_orca_output(o_file)
    if 'gibbs free energy' not in terms:
        print(f'No entropy in "{o_file}"')
        search_terms = {'TS_trans': 'Translational entropy'}
        trans_entr = parse_orca_output(o_file, search_terms=search_terms)
        if 'TS_trans' in trans_entr:
            print(f'Found translational entropy')
            terms['G'] = terms['H'] - trans_entr['TS_trans']
            terms['G-E(el)'] = terms['G'] - terms['E_sp']


    calcs[o_file[:-4]] = terms


raw_res_df = pd.DataFrame(calcs)
raw_res_df.T

No entropy in "Step4/[Cl_CH3_Br]-.out"
Found translational entropy
No entropy in "Step3/Cl-_CH3Br.out"
No entropy in "Step2/Cl-_CH3Br.out"
Found translational entropy
No entropy in "Step2/CH3Cl_Br-.out"
Found translational entropy
No entropy in "Step5/CH3Br.out"
No entropy in "Step5/Cl-_CH3Br.out"
No entropy in "Step5/Cl-.out"
No entropy in "Step5/Br-.out"
No entropy in "Step5/CH3Cl_Br-.out"
No entropy in "Step5/[Cl_CH3_Br]-.out"
No entropy in "Step5/CH3Cl.out"
No entropy in "Step6/Cl-_CH3Br.out"
No entropy in "Step6/CH3Cl_Br-.out"
No entropy in "Step6/[Cl_CH3_Br]-.out"
No entropy in "Step1/CH3Br.out"
Found translational entropy
No entropy in "Step1/Cl-.out"
Found translational entropy
No entropy in "Step1/Br-.out"
Found translational entropy
No entropy in "Step1/CH3Cl.out"
Found translational entropy


,E_sp,E_therm,H,TS,G,G-E(el)
Step4/[Cl_CH3_Br]-,-3073.632714,-3073.591893,-3073.590949,0.034066,-3073.610196,0.022518
Step3/Cl-_CH3Br,-3073.651568,NaN,NaN,NaN,NaN,NaN
Step2/Cl-_CH3Br,-3073.635520,-3073.593542,-3073.592597,0.035411,-3073.611844,0.023675
Step2/CH3Cl_Br-,-3073.652284,-3073.609444,-3073.608500,0.035537,-3073.627747,0.024537
Step5/CH3Br,-2613.942478,NaN,NaN,NaN,NaN,NaN
Step5/Cl-_CH3Br,-3074.191893,NaN,NaN,NaN,NaN,NaN
Step5/Cl-,-460.234317,NaN,NaN,NaN,NaN,NaN
Step5/Br-,-2574.129170,NaN,NaN,NaN,NaN,NaN
Step5/CH3Cl_Br-,-3074.202621,NaN,NaN,NaN,NaN,NaN
Step5/[Cl_CH3_Br]-,-3074.185276,NaN,NaN,NaN,NaN,NaN


In [188]:
step1_corr = ['Br-', 'Cl-', 'CH3Br', 'CH3Cl'] 
step2_corr = ['CH3Cl_Br-', 'Cl-_CH3Br'] 
step4_corr = ['[Cl_CH3_Br]-']

for stuff in step1_corr:
    raw_res_df[f'Step5/{stuff}']['G'] = raw_res_df[f'Step5/{stuff}']['E_sp'] - raw_res_df[f'Step1/{stuff}']['G-E(el)']
for stuff in step2_corr:
    raw_res_df[f'Step5/{stuff}']['G'] = raw_res_df[f'Step5/{stuff}']['E_sp'] - raw_res_df[f'Step2/{stuff}']['G-E(el)']
    raw_res_df[f'Step6/{stuff}']['G'] = raw_res_df[f'Step6/{stuff}']['E_sp'] - raw_res_df[f'Step2/{stuff}']['G-E(el)']
for stuff in step4_corr:
    raw_res_df[f'Step5/{stuff}']['G'] = raw_res_df[f'Step5/{stuff}']['E_sp'] - raw_res_df[f'Step4/{stuff}']['G-E(el)']
    raw_res_df[f'Step6/{stuff}']['G'] = raw_res_df[f'Step6/{stuff}']['E_sp'] - raw_res_df[f'Step4/{stuff}']['G-E(el)']

In [189]:
res_df = raw_res_df.filter(regex='Step[1,2,4,5,6]') * 627.5094740631
res_df.T

,E_sp,E_therm,H,TS,G,G-E(el)
Step4/[Cl_CH3_Br]-,-1.928734e+06,-1.928708e+06,-1.928707e+06,21.377039,-1.928720e+06,14.130330
Step2/Cl-_CH3Br,-1.928735e+06,-1.928709e+06,-1.928708e+06,22.221014,-1.928721e+06,14.856305
Step2/CH3Cl_Br-,-1.928746e+06,-1.928719e+06,-1.928718e+06,22.300043,-1.928731e+06,15.397211
Step5/CH3Br,-1.640274e+06,NaN,NaN,NaN,-1.640288e+06,NaN
Step5/Cl-_CH3Br,-1.929085e+06,NaN,NaN,NaN,-1.929099e+06,NaN
Step5/Cl-,-2.888014e+05,NaN,NaN,NaN,-2.887937e+05,NaN
Step5/Br-,-1.615290e+06,NaN,NaN,NaN,-1.615282e+06,NaN
Step5/CH3Cl_Br-,-1.929091e+06,NaN,NaN,NaN,-1.929107e+06,NaN
Step5/[Cl_CH3_Br]-,-1.929080e+06,NaN,NaN,NaN,-1.929095e+06,NaN
Step5/CH3Cl,-3.137927e+05,NaN,NaN,NaN,-3.138074e+05,NaN


In [190]:
res_df['Step1/Cl- + CH3Br'] = res_df['Step1/Cl-'] + res_df['Step1/CH3Br']
res_df['Step1/CH3Cl + Br-'] = res_df['Step1/Br-'] + res_df['Step1/CH3Cl']
res_df['Step5/Cl- + CH3Br'] = res_df['Step5/Cl-'] + res_df['Step5/CH3Br']
res_df['Step5/CH3Cl + Br-'] = res_df['Step5/Br-'] + res_df['Step5/CH3Cl']
res_df.T

,E_sp,E_therm,H,TS,G,G-E(el)
Step4/[Cl_CH3_Br]-,-1.928734e+06,-1.928708e+06,-1.928707e+06,21.377039,-1.928720e+06,14.130330
Step2/Cl-_CH3Br,-1.928735e+06,-1.928709e+06,-1.928708e+06,22.221014,-1.928721e+06,14.856305
Step2/CH3Cl_Br-,-1.928746e+06,-1.928719e+06,-1.928718e+06,22.300043,-1.928731e+06,15.397211
Step5/CH3Br,-1.640274e+06,NaN,NaN,NaN,-1.640288e+06,NaN
Step5/Cl-_CH3Br,-1.929085e+06,NaN,NaN,NaN,-1.929099e+06,NaN
Step5/Cl-,-2.888014e+05,NaN,NaN,NaN,-2.887937e+05,NaN
Step5/Br-,-1.615290e+06,NaN,NaN,NaN,-1.615282e+06,NaN
Step5/CH3Cl_Br-,-1.929091e+06,NaN,NaN,NaN,-1.929107e+06,NaN
Step5/[Cl_CH3_Br]-,-1.929080e+06,NaN,NaN,NaN,-1.929095e+06,NaN
Step5/CH3Cl,-3.137927e+05,NaN,NaN,NaN,-3.138074e+05,NaN


In [191]:
res_df.filter(regex='Step[1,2,5]').T['G'].sort_values()

Step5/CH3Cl_Br-      -1.929107e+06
Step5/Cl-_CH3Br      -1.929099e+06
Step5/[Cl_CH3_Br]-   -1.929095e+06
Step5/CH3Cl + Br-    -1.929089e+06
Step5/Cl- + CH3Br    -1.929081e+06
Step2/CH3Cl_Br-      -1.928731e+06
Step1/CH3Cl + Br-    -1.928727e+06
Step2/Cl-_CH3Br      -1.928721e+06
Step1/Cl- + CH3Br    -1.928709e+06
Step5/CH3Br          -1.640288e+06
Step1/CH3Br          -1.640032e+06
Step5/Br-            -1.615282e+06
Step1/Br-            -1.615082e+06
Step5/CH3Cl          -3.138074e+05
Step1/CH3Cl          -3.136449e+05
Step5/Cl-            -2.887937e+05
Step1/Cl-            -2.886768e+05
Name: G, dtype: float64

We see that the the lowest energy state before the transition state is $Cl^-\;-\; CH_3Br$ with -3073.611844 Ha

In [192]:
svp_ref = res_df['Step2/Cl-_CH3Br']
svp_ref

E_sp      -1.928735e+06
E_therm   -1.928709e+06
H         -1.928708e+06
TS         2.222101e+01
G         -1.928721e+06
G-E(el)    1.485630e+01
Name: Step2/Cl-_CH3Br, dtype: float64

In [193]:
(res_df.T - svp_ref)['G'].filter(regex='[1,2,4,5,6].*[_,+]')

Step4/[Cl_CH3_Br]-      1.034550
Step2/Cl-_CH3Br         0.000000
Step2/CH3Cl_Br-        -9.978894
Step5/Cl-_CH3Br      -378.842386
Step5/CH3Cl_Br-      -386.115362
Step5/[Cl_CH3_Br]-   -373.964046
Step6/Cl-_CH3Br      -436.976945
Step6/CH3Cl_Br-      -443.777403
Step6/[Cl_CH3_Br]-   -426.961501
Step1/Cl- + CH3Br      11.907337
Step1/CH3Cl + Br-      -6.402661
Step5/Cl- + CH3Br    -360.717402
Step5/CH3Cl + Br-    -368.910517
Name: G, dtype: float64

In [194]:
tzvpd_ref = res_df['Step5/Cl-_CH3Br']
tzvpd_ref
(res_df.T - tzvpd_ref)['G'].filter(regex='[5].*[_,+]')

Step5/Cl-_CH3Br        0.000000
Step5/CH3Cl_Br-       -7.272975
Step5/[Cl_CH3_Br]-     4.878340
Step5/Cl- + CH3Br     18.124985
Step5/CH3Cl + Br-      9.931869
Name: G, dtype: float64

In [195]:
latex_replacements = {'Step1/Br-': r'\ce{Br-}',
                      'Step5/Br-': r'\ce{Br-}',
                      'Step1/CH3Br': r'\ce{CH3Br}',
                      'Step5/CH3Br': r'\ce{CH3Br}',
                      'Step1/CH3Cl': r'\ce{CH3Cl}',
                      'Step5/CH3Cl': r'\ce{CH3Cl}',
                      'Step1/CH3Cl + Br-': r'\ce{CH3Cl + Br-}',
                      'Step5/CH3Cl + Br-': r'\ce{CH3Cl + Br-}',
                      'Step1/Cl-': r'\ce{Cl-}',
                      'Step5/Cl-': r'\ce{Cl-}',
                      'Step1/Cl- + CH3Br': r'\ce{Cl- + CH3Br}',
                      'Step5/Cl- + CH3Br': r'\ce{Cl- + CH3Br}',
                      'Step2/CH3Cl\_Br-': r'\ce{CH3Cl\bond{...}Br-}',
                      'Step5/CH3Cl\_Br-': r'\ce{CH3Cl\bond{...}Br-}',
                      'Step6/CH3Cl\_Br-': r'\ce{CH3Cl\bond{...}Br-}',
                      'Step2/Cl-\_CH3Br': r'\ce{Cl^-\bond{...}CH3Br}',
                      'Step5/Cl-\_CH3Br': r'\ce{Cl^-\bond{...}CH3Br}',
                      'Step6/Cl-\_CH3Br': r'\ce{Cl^-\bond{...}CH3Br}',
                      'Step4/[Cl\_CH3\_Br]-': r'\ce{[Cl\bond{...}CH3\bond{...}Br]-}',
                      'Step5/[Cl\_CH3\_Br]-': r'\ce{[Cl\bond{...}CH3\bond{...}Br]-}',
                      'Step6/[Cl\_CH3\_Br]-': r'\ce{[Cl\bond{...}CH3\bond{...}Br]-}',
                     }

sorted_latex_keys = sorted(latex_replacements.keys(), reverse=True)

In [196]:
table = res_df.filter(regex='Step[1,2,4,5,6]').T[['E_sp', 'H', 'G', 'G-E(el)']].sort_index().to_latex(float_format="%.2f", na_rep='-')
for key in sorted_latex_keys:
    table = table.replace(key, latex_replacements[key])
    
print(table)

\begin{tabular}{lrrrr}
\toprule
{} &        E\_sp &           H &           G &  G-E(el) \\
\midrule
\ce{Br-}          & -1615073.63 & -1615070.37 & -1615082.01 &    -8.38 \\
\ce{CH3Br}        & -1640045.72 & -1640020.05 & -1640031.85 &    13.87 \\
\ce{CH3Cl}        &  -313659.67 &  -313633.71 &  -313644.94 &    14.73 \\
\ce{CH3Cl + Br-}  & -1928733.30 & -1928704.08 & -1928726.95 &     6.34 \\
\ce{Cl-}          &  -288669.14 &  -288665.88 &  -288676.80 &    -7.66 \\
\ce{Cl- + CH3Br}  & -1928714.85 & -1928685.93 & -1928708.64 &     6.21 \\
\ce{CH3Cl\bond{...}Br-}    & -1928745.93 & -1928718.45 & -1928730.53 &    15.40 \\
\ce{Cl^-\bond{...}CH3Br}    & -1928735.41 & -1928708.47 & -1928720.55 &    14.86 \\
\ce{[Cl\bond{...}CH3\bond{...}Br]-} & -1928733.65 & -1928707.44 & -1928719.52 &    14.13 \\
\ce{Br-}          & -1615290.44 &           - & -1615282.06 &        - \\
\ce{CH3Br}        & -1640273.67 &           - & -1640287.54 &        - \\
\ce{CH3Cl}        &  -313792.68 &           - & 

In [199]:
res_df.filter(regex='Step6').T

,E_sp,E_therm,H,TS,G,G-E(el)
Step6/Cl-_CH3Br,-1.929143e+06,NaN,NaN,NaN,-1.929158e+06,NaN
Step6/CH3Cl_Br-,-1.929149e+06,NaN,NaN,NaN,-1.929164e+06,NaN
Step6/[Cl_CH3_Br]-,-1.929133e+06,NaN,NaN,NaN,-1.929148e+06,NaN


In [198]:
dmf_ref = res_df['Step6/Cl-_CH3Br']
dmf_ref
(res_df.T - dmf_ref)['G'].filter(regex='Step[6]')

Step6/Cl-_CH3Br        0.000000
Step6/CH3Cl_Br-       -6.800458
Step6/[Cl_CH3_Br]-    10.015444
Name: G, dtype: float64

In [216]:
# Eyring equation
def eyring(dG, T=297):
    kB = 1.380649e-23
    h = 6.626068e-34
    kappa = 1
    R = 8.31446261815324
    
    return kappa * (kB * T / h) * np.exp(-dG/(R*T))

eyring(10.015444 *  4186.798188)


261130.4190484405

In [220]:
eyring(17.7 * 4186.798188)

0.10522309956785651

In [218]:
eyring(10.015444 * 4186.798188)/eyring(17.7 * 4186.798188)

455418.7780017808